In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

In [2]:
if torch.cuda.is_available():
    print("CUDA is available")
else:
    print("CUDA is not available")

CUDA is available


In [3]:
import pandas as pd
import numpy as np

In [4]:
malware_calls = pd.read_csv("../datasets/CatakPreprocessed.csv")

In [5]:
malware_calls.iloc[0]['class']

'Trojan'

In [6]:
malware_calls.head()

,api,class
0,ldrloaddll ldrgetprocedureaddress regopenkeyex...,Trojan
1,getsystemtimeasfiletime ntallocatevirtualmemor...,Trojan
2,ldrgetdllhandle ldrgetprocedureaddress getsyst...,Backdoor
3,ldrloaddll ldrgetprocedureaddress regopenkeyex...,Backdoor
4,ldrloaddll ldrgetprocedureaddress wsastartup n...,Trojan


In [7]:
malware_calls['class'].value_counts()

Trojan        1001
Backdoor      1001
Downloader    1001
Worms         1001
Virus         1001
Dropper        891
Spyware        832
Adware         379
Name: class, dtype: int64

In [8]:
num_classes = len(malware_calls["class"].value_counts())
class_weights = (1 - (malware_calls['class'].value_counts().sort_index() / len(malware_calls))).values
class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.9467, 0.8592, 0.8592, 0.8746, 0.8829, 0.8592, 0.8592, 0.8592],
       device='cuda:0')

### Class Mapping

In [9]:
CAT2IDX = {
    'Virus': 0,
    'Trojan': 1,
    'Worms': 2,
    'Downloader': 3,
    'Backdoor': 4,
    'Dropper': 5,
    'Spyware': 6,
    'Adware': 7,
}

IDX2CAT = {
    0:'Virus',
    1:'Trojan',
    2:'Worms',
    3:'Downloader',
    4:'Backdoor',
    5:'Dropper',
    6:'Spyware',
    7:'Adware',
}

from huggingface_hub import login
login()

## Load DistilBERT Model Checkpoint from Hugging Face

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, label2id=CAT2IDX, id2label=IDX2CAT)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
tokenizer.model_max_length = 512

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Split Data for Training and Validation

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(malware_calls.api, malware_calls['class'],
test_size=0.2, random_state=75, stratify = malware_calls['class'])

In [11]:
from datasets import load_dataset, Dataset
import datasets

train = Dataset.from_pandas(pd.concat([Y_train, X_train], axis=1)).remove_columns('__index_level_0__')
validation = Dataset.from_pandas(pd.concat([Y_test, X_test], axis=1)).remove_columns('__index_level_0__')

dataset = datasets.DatasetDict({"train": train, "validation": validation})
dataset["train"] = dataset["train"].rename_column("class", "label")
dataset["validation"] = dataset["validation"].rename_column("class", "label")

# Rename 'api' to 'text' in both train and validation datasets
dataset["train"] = dataset["train"].rename_column("api", "text")
dataset["validation"] = dataset["validation"].rename_column("api", "text")
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5685
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1422
    })
})

In [12]:
def convert_label(example):
    # Convert the label using the CAT2IDX mapping
    example['label'] = CAT2IDX[example['label']]
    return example

# Apply the function to both the train and validation datasets
dataset['train'] = dataset['train'].map(convert_label)
dataset['validation'] = dataset['validation'].map(convert_label)

Map:   0%|          | 0/5685 [00:00<?, ? examples/s]

Map:   0%|          | 0/1422 [00:00<?, ? examples/s]

## Load or Create Tokenized Dataset

In [13]:
from datasets import load_from_disk
import os
from transformers import AutoTokenizer

# Assuming you have already defined your tokenizer
# If not, initialize it like this (replace 'your-model-name' with the model you are using):
# tokenizer = AutoTokenizer.from_pretrained('your-model-name')

def tokenize_function(examples):
    # Extract text
    text = examples['text']
    
    # Set truncation side (optional, based on your preference)
    tokenizer.truncation_side = "left"

    # Tokenize with truncation and padding
    tokenized_inputs = tokenizer(
        text,
        padding="max_length",  # Add padding to the max_length
        truncation=True,
        max_length=512,
        return_tensors='np'   # Return numpy tensors
    )
    
    return tokenized_inputs

# Rest of your code for loading/saving dataset
directory_path = 'tokenized_datasets'
file_name = 'catak_tokenized'
full_path = os.path.join(directory_path, file_name)

if os.path.exists(full_path):
    # Load the existing dataset
    tokenized_dataset = load_from_disk(full_path)
    print("Loaded the tokenized dataset.")
else:
    # Ensure the directory exists
    os.makedirs(directory_path, exist_ok=True)
    
    # Add pad token if it's not already present
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": "[PAD]"})
        # Resize model embeddings to account for new tokens
        model.resize_token_embeddings(len(tokenizer))

    # Apply tokenization function to the dataset
    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Save the new tokenized dataset
    tokenized_dataset.save_to_disk(full_path)
    print("Saved new tokenized dataset.")

# Print or return the tokenized dataset
print(tokenized_dataset)

Loaded the tokenized dataset.
DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 5685
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1422
    })
})


In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

NameError: name 'tokenizer' is not defined

In [ ]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import roc_auc_score
from scipy.special import softmax

def compute_metrics(eval_pred):
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    acc = load_metric("accuracy")
    mcc = load_metric("matthews_correlation")
    #auc = load_metric("auc")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision.compute(predictions=predictions, average = "macro", references=labels)["precision"]
    recall = recall.compute(predictions=predictions, average = "macro", references=labels)["recall"]
    f1 = f1.compute(predictions=predictions, average = "macro", references=labels)["f1"]
    acc = acc.compute(predictions=predictions, references=labels)["accuracy"]
    mcc = mcc.compute(predictions=predictions, references=labels)["matthews_correlation"]
    auc = roc_auc_score(labels, softmax(logits, axis=1), multi_class='ovo', average='macro')
    return {"precision": precision, "recall": recall, "acc": acc, "mcc": mcc, "f1": f1, "auc":auc}

In [17]:
import torch
torch.cuda.empty_cache()
# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of GPUs available
    n_gpu = torch.cuda.device_count()
    for i in range(n_gpu):
        print(f"GPU {i}:")
        print(f"  Total memory: {torch.cuda.get_device_properties(i).total_memory / 1e9} GB")
        print(f"  Allocated memory: {torch.cuda.memory_allocated(i) / 1e9} GB")
        print(f"  Cached memory: {torch.cuda.memory_reserved(i) / 1e9} GB")
else:
    print("CUDA is not available.")

GPU 0:
  Total memory: 8.58947584 GB
  Allocated memory: 5.12e-07 GB
  Cached memory: 0.002097152 GB


### Prediction Using Base Model Performance is not Ideal

In [18]:
input = tokenizer.encode(malware_calls.iloc[0]['api'], return_tensors="pt")
input = input.to(torch.device("cuda"))
logits = model(input).logits
prediction = torch.argmax(logits)
print(malware_calls.iloc[0]['class'] + " - " + IDX2CAT[prediction.tolist()])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [20]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=4,
    lora_alpha=32,
    lora_dropout=0.01,
    target_modules=['q_lin', 'v_lin'] # Only apply LORA to the query and value projections, the paper on LORA suggests that this provides the best results
)

In [21]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### PEFT Will Only Tune %0.06 of the Parameters

In [22]:
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 633,608 || all params: 67,593,232 || trainable%: 0.9373837901404093


In [31]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 100

In [32]:
from transformers import TrainingArguments

# define training arguments
training_args = TrainingArguments(
    output_dir= model_name + "-Malware-Classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased"
model_path = model_name + "-Malware-Classification"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

OSError: distilbert-base-uncased-Malware-Classification does not appear to have a file named config.json. Checkout 'https://huggingface.co/distilbert-base-uncased-Malware-Classification/None' for available files.

In [33]:
from transformers import Trainer

try:
    # creater trainer object
    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
        compute_metrics=compute_metrics,
    )
    
    trainer.train()
    
    
except RuntimeError as e:
    print(f"An error occurred: {e}")
    torch.cuda.empty_cache()  # Clearing the CUDA cache

  0%|          | 0/142200 [00:00<?, ?it/s]

{'loss': 1.9052, 'learning_rate': 0.0009964838255977497, 'epoch': 0.35}
{'loss': 1.9164, 'learning_rate': 0.0009929676511954994, 'epoch': 0.7}


  0%|          | 0/356 [00:00<?, ?it/s]

c:\Users\regis\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 1.8606908321380615, 'eval_precision': 0.2926126900872718, 'eval_recall': 0.2799062530340609, 'eval_acc': 0.2890295358649789, 'eval_mcc': 0.19364218153333052, 'eval_f1': 0.2572794935217092, 'eval_auc': 0.6890314173086913, 'eval_runtime': 29.2505, 'eval_samples_per_second': 48.615, 'eval_steps_per_second': 12.171, 'epoch': 1.0}
{'loss': 1.9003, 'learning_rate': 0.000989451476793249, 'epoch': 1.05}
{'loss': 1.9173, 'learning_rate': 0.0009859353023909986, 'epoch': 1.41}
{'loss': 1.9064, 'learning_rate': 0.0009824191279887483, 'epoch': 1.76}


...

{'eval_loss': 1.5610990524291992, 'eval_precision': 0.6067347478949163, 'eval_recall': 0.598045498751123, 'eval_acc': 0.5879043600562588, 'eval_mcc': 0.5259325938400049, 'eval_f1': 0.6013933782305144, 'eval_auc': 0.8729865288452804, 'eval_runtime': 16.79, 'eval_samples_per_second': 84.693, 'eval_steps_per_second': 21.203, 'epoch': 98.0}
{'loss': 0.6231, 'learning_rate': 1.89873417721519e-05, 'epoch': 98.1}
{'loss': 0.627, 'learning_rate': 1.547116736990155e-05, 'epoch': 98.45}
{'loss': 0.6549, 'learning_rate': 1.1954992967651194e-05, 'epoch': 98.8}


  0%|          | 0/356 [00:00<?, ?it/s]

{'eval_loss': 1.554926872253418, 'eval_precision': 0.6047222519817455, 'eval_recall': 0.5949037559209938, 'eval_acc': 0.5857946554149086, 'eval_mcc': 0.5234002979983099, 'eval_f1': 0.5986617317788157, 'eval_auc': 0.8730731281612797, 'eval_runtime': 16.1538, 'eval_samples_per_second': 88.029, 'eval_steps_per_second': 22.038, 'epoch': 99.0}
{'loss': 0.6633, 'learning_rate': 8.438818565400844e-06, 'epoch': 99.16}
{'loss': 0.6443, 'learning_rate': 4.922644163150493e-06, 'epoch': 99.51}
{'loss': 0.6182, 'learning_rate': 1.4064697609001406e-06, 'epoch': 99.86}


  0%|          | 0/356 [00:00<?, ?it/s]

{'eval_loss': 1.555328607559204, 'eval_precision': 0.603934440043556, 'eval_recall': 0.5969301408502801, 'eval_acc': 0.5879043600562588, 'eval_mcc': 0.5258677780142589, 'eval_f1': 0.5993309151927436, 'eval_auc': 0.8731178782697037, 'eval_runtime': 17.0582, 'eval_samples_per_second': 83.361, 'eval_steps_per_second': 20.87, 'epoch': 100.0}
{'train_runtime': 16031.1187, 'train_samples_per_second': 35.462, 'train_steps_per_second': 8.87, 'train_loss': 1.2234708714652833, 'epoch': 100.0}


In [ ]:
input = tokenizer.encode(malware_calls.iloc[0]['api'], return_tensors="pt")
input = input.to(torch.device("cuda"))
logits = peft_model(input).logits
prediction = torch.argmax(logits)
print(malware_calls.iloc[0]['class'] + " - " + IDX2CAT[prediction.tolist()])

Trojan - Downloader


In [ ]:
input = tokenizer.encode(malware_calls.iloc[0]['api'], return_tensors="pt")
input = input.to(torch.device("cuda"))
logits = peft_model(input).logits
prediction = torch.max(logits, 1).indices
print(malware_calls.iloc[0]['class'] + " - " + IDX2CAT[prediction.tolist()[0]])

In [ ]:
print("hello")